In [29]:
#Basics
import pandas as pd
import numpy as np
import itertools

#Time series 
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

# Ignoring warning messages from python
import logging, sys
logging.disable(logging.INFO)
# logging.disable(sys.maxsize)

import warnings
warnings.filterwarnings('ignore')

#Modules
import wrangle as wr

<div class="alert alert-info"><b>Model Evaluation:  These models will be evalutated using root-mean-square errorr (RMSE) and mean absolute percentage error (MAPE) on a 1 day and 3 day basis.</b>
    <ul>
        <li><b>RMSE</b> allows us to see the overall error of the model in the energy load units (MW).</li>
        <li><b>MAPE</b> complements that by telling us how accurate the model is on a percentage basis.  This is important as the energy load varies greatly throughout the year, so a 1000 MW RMSE on a hot summer day will have a much lower MAPE percentage than a 1000 MW RMSE on a comfortable Saturday in the Fall.</li>
        <li>The <b>3 day metrics</b> will tell us the overall performance of the 3 day forecast.  However, we expect the 3rd day to be less accurate than the 1st day and production models will be re-run on a daily or sub-daily basis. For that reason, we also will look at <b>1 day metrics</b> to see the model performance over the first day only.</li>
    </ul></div>

<div class="alert alert-info"><b>Cross-Validation Notes:</b><br><br>Cross validation was performed on a sliding window.  Models were trained on <b>4 year initial windows</b>, with a <b>forecasting horizon of 3 days</b>.  Due to the large amount of data we had available, we minimized processing time by using a <b>36 day sliding offset</b>.  A 36 day offset ensured that our forecasting window started at different days of the week and rotated through different parts of the month. <br><br> The downside to this approach is that it is possible the model is not tested over more anomalous time periods (such as holidays or major weather events like Hurrican Harvey).  A smaller sliding offset (such as 1 or 2 days) would catch those time periods, however it would require much more processing time. </div>

### Prophet Models

**Note:** You will notice that we re-acquire the dataframes in this section.  Prophet requires time zone naive inputs, however we wanted our explore section above to display CST time zones for readability.

<div class="alert alert-danger"><b>Final Report Prep:  Make sure those two get prophet functions (seen in next cell) are in the main wrangle file.  I have not added them yet as I'm not sure if Greg has all his changes in there or not. - Cayt</b></div>

In [14]:
#Acquire data
dfr = wr.get_prophet_df_w_meantemp()
#Filter train based on CST time (EOY 2017, time zone naive UTC time is in this df)
trainr = dfr[dfr.ds < '2018-01-01 06:00:00']
testr = dfr[dfr.ds > '2014-01-01 04:00:00']

#Create df w/o regression column
train = trainr[['ds','y']].copy()

In [4]:
#Set metrics to evaluate
metrics = ['rmse','mape']

##### Basic Prophet model

<div class='alert alert-success'>Uses default Prophet settings:
<ul>
    <li>Auto for all seasonality</li>
    <li>No holidays</li>
    <li>No regression</li>
    </ul>
    </div>

In [5]:
#Create and Fit Basic Prophet model
model = Prophet().fit(train)

#Create cross-validation matrix
df_cv = cross_validation(model,initial='1461 days', period='36 days', horizon = '3 days',disable_tqdm=True)

#Get 1 and 3 day model performance
df_p_1d = performance_metrics(df_cv,rolling_window=.33,metrics=metrics)
df_p_3d = performance_metrics(df_cv,rolling_window=1,metrics=metrics)

In [6]:
#print model results
wr.print_model_results('Basic Prophet',df_p_1d,df_p_3d)

Basic Prophet model performance:
1 day rmse: 1220.0 MW
1 day mape: 8.1%
3 day rmse: 1302.0 MW
3 day mape: 8.5%


<div class="alert alert-danger"><b>Final Report Prep:  Need to add this new function to wrangle, then update the above line to reference wrangle.</b></div>

<div class='alert alert-success'><b>Seasonality Notes:</b>
<ul>
    <li>The auto seasonality set a scale of 10 for the yearly, weekly and daily seasons.  Based off our industry knowledge and exploration of the data, we expect the daily and weekly seasonalities to carry a higher weight than the yearly seasonality.  In addition, we are concerned that yearly seasonality is a proxy for a temperature regressor and that including both may be detrimental to the model.</li>
    <li>The yearly fourier order of 10 seemed quite high compared to the weekly order of 3 and daily order of 4.</li>
    <li>We want all seasonalities to be additive.</li>
    </ul>
</div>

##### Prophet Model v2 - Hyperparameter first pass 


<div class='alert alert-success'>Prophet settings:
<ul>
    <li>Remove auto-seasonality</li>
    <li>Manually add seasonality:
    <ul>
        <li>daily fourier order of 6, with a scale of 30</li>
        <li>weekly fourier order of 6, with a scale of 30</li>
        <li>yearly fourier order of 6, with a scale of 10</li>
        <li>NOTES: choose a higher daily and weekly order and scale, while lowering yearly scale.
        </ul>
    </li>
    <li>Added US Federal observed holidays</li>
    <li>Added mean temperature regressor</li>
    </ul>
    </div>

In [7]:
#Get US holidays
only_holidays = wr.get_holiday_df(trainr)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
#Create and fit model

#Create model
m = Prophet(yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            holidays=only_holidays)
#Add Seasonality
m = m.add_seasonality(name='daily', 
                      period=1, 
                      fourier_order=6,
                      prior_scale=30
                     )

m = m.add_seasonality(name='weekly', 
                      period=7, 
                      fourier_order=6,
                      prior_scale=30
                     )
m = m.add_seasonality(name='yearly', 
                      period=365.25, 
                      fourier_order=6,
                      prior_scale=10
                     )
#Add Regressor
m = m.add_regressor('mean_temp')

#fit model
m.fit(trainr)

#Perform cross-validation
df_cv_v2 = cross_validation(m, initial='1461 days', period='36 days', horizon = '3 days') 
#Get performance metrics
df_v2_1d = performance_metrics(df_cv_v2, rolling_window=.33, metrics=metrics)
df_v2_3d = performance_metrics(df_cv_v2, rolling_window=1, metrics=metrics)


  0%|          | 0/41 [00:00<?, ?it/s]

TypeError: print_model_results() got an unexpected keyword argument 'disable_tqdm'

In [9]:
#print model results
wr.print_model_results('Prophet - First Hyperparameter Pass',df_v2_1d,df_v2_3d)

Prophet - First Hyperparameter Pass model performance:
1 day rmse: 1191.0 MW
1 day mape: 8.0%
3 day rmse: 1279.0 MW
3 day mape: 8.4%


##### Prophet post-Hyperparameter tuning

**Note:** While we would have preferred to run more hyperparameter combinations all at once, we were limited by the processing time.  We chose to do smaller batches of hyperparameters so that we could perform more iterations while still maximizing the combinations.

<div class='alert alert-info'><b>Hyperparameter Tuning - Round 1 - 48 models</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Remove yearly seasonality as it is a proxy for the temperature regressor. We suspect the two are not independent.</li>
        <li>Keep the mean temperature as a regressor.</li>
        <li>Try multiple scales [10,20,30] and fourier orders [2,3] for daily and weekly seasonalities</li>
        </ul>

<br><b>Results:</b> 
<ul>
    <li>This performed significantly worse than the first pass, with the highest performing model having a 3 day MAPE of 13.2%. <b>Action:</b> Re-add in the yearly seasonality</li>
    <li>Seasonal scales of 10 across the board performed the best - which may be due to using the default holiday and regressor scale.  <b>Action:</b> Go with lower seasonal scales. </li>
    <li>The top performing models all had daily and weekly fourier orders of 3. <b>Action:</b> Try fourier orders of 3+.</li>
    </ul>
</div>

<div class='alert alert-info'><b>Hyperparameter Tuning - Round 2 - 36 models</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Added yearly seasonality with scale of 5 and 10</li>
        <li>Use a daily and weekly scale of 10</li>
        <li>Use daily and yearly fourier orders of 3-5, and a weekly fourier order of 3 and 4.</li>
        </ul>

<br><b>Results:</b> 
<ul>
    <li>Significant improvement in 3 day MAPE from the first round of models, with the best at 8.2%. <b>Action:</b> Keep yearly seasonality</li>
    <li>Top models contained a daily fourier order of 5. <b>Action:</b> Try 5+ in next round. </li>
    <li>Top models contained a yearly fourier order of 3. <b>Action:</b> Try 2 and 3 in next round. </li>
    <li>Mixed results on yearly scale options of 5 and 10 and weekly order of 3 and 4. Slight edge for a scale of 5 and order of 3.</li>
    </ul>
</div>

<div class='alert alert-info'><b>Hyperparameter Tuning - Round 3 - 12 models</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Keep daily, weekly, yearly scales of 10, 10 and 5, respectively.  Try regressor scales of 10 and 15.</li>
        <li>Use a daily fourier order of 5-7</li>
        <li>Use yearly fourier order of 2 and 3.</li>
        </ul>

<br><b>Results:</b> 
<ul>
    <li>Yearly fourier order of 3 consistently outperformed. <b>Action:</b> Keep yearly at 3.</li>
    <li>Regressor scales of 10 mostly outperformed scales of 15. <b>Action:</b> Keep regressor scale of 10. </li>
    <li>Mixed results on daily fourier order, however 5 performed best in situations with the last two bullets. <b>Action:</b> Keep daily order of 5</li>
    </ul>
</div>

<div class='alert alert-info'><b>Hyperparameter Tuning - Round 4 - 24 models</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Try smaller scale changes of 8 and 10 for daily, 8, 10 and 12 for weekly and 5 and 7 for yearly. Keep regressor at 10.</li>
        <li>Use a daily fourier order of 5 and a yearly fourier order of 3</li>
        <li>Use weekly fourier order of 3 and 5.</li>
        </ul>

<br><b>Results:</b> 
<ul>
    <li>Weekly fourier order of 5 consistently outperformed. <b>Action:</b> Potentially try 5+.</li>
    <li>Yearly scale of 7 consistently outperformed <b>Action:</b> Keep yearly scale of 7, consider trying 10 again on final round since it performed very closesly to 5 in the past. </li>
    <li>Given the above bullets, daily and weekly scales of 10 performed best. <b>Action:</b> Keep at 10</li>
    </ul>
</div>

<div class='alert alert-info'><b>Hyperparameter Tuning - Round 5 (Final) - 12 models</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Daily, weekly and regressor scales at 10. Yearly scales at 7 and 10.</li>
        <li>Daily fourier order of 5, yearly fourier order of 3.  Weekly fourier order of 5-10.</li>
        <li>US Federal Observed holidays</li>
        <li>Mean Temperature regressor</li>
        </ul>

<br><b>Results:</b> 
<ul>
    <li></li>
    <li></li>
    <li></li>
    </ul>
</div>

<div class='alert alert-success'><b>Top Performing Model</b>
<br><br><u>Hyperparameters:</u>
    <ul>
        <li>Daily, weekly and regressor scales at 10. Yearly scales at <b>???</b>.</li>
        <li>Daily fourier order of 5, yearly fourier order of 3.  Weekly fourier order of <b>???</b></li>
        <li>US Federal Observed holidays</li>
        <li>Mean Temperature regressor</li>
        </ul>
</div>

In [ ]:
#Create the model
m = Prophet(yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            holidays=only_holidays)
#Add seasonalities
m = m.add_seasonality(name='daily', 
                      period=1, 
                      fourier_order=5,
                      prior_scale=10
                     )
m = m.add_seasonality(name='weekly', 
                      period=7, 
                      fourier_order=,
                      prior_scale=10
                     )
m = m.add_seasonality(name='yearly', 
                      period=365.25, 
                      fourier_order=3,
                      prior_scale=
                     )
#Add Regressors
m = m.add_regressor('mean_temp')

#fit model
m.fit(trainr)

#Perform cross-validation
df_cv_final = cross_validation(m, initial='1461 days', period='36 days', horizon = '3 days',disable_tqdm=True) 
#Get performance metrics
df_final_1d = performance_metrics(df_cv_final, rolling_window=.33, metrics=metrics)
df_final_3d = performance_metrics(df_cv_final, rolling_window=1, metrics=metrics)

#print model results
wr.print_model_results('Tuned Prophet Model',df_final_1d,df_final_3d)

### Run Model on Test Dataset

In [15]:
testr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74472 entries, 35063 to 109534
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         74472 non-null  datetime64[ns]
 1   y          74472 non-null  float64       
 2   mean_temp  74472 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.3 MB


In [30]:
# Initialize performance dataframe
baseline_test_df_1d = pd.DataFrame(columns=['rmse', 'mape'])
baseline_test_df_3d = pd.DataFrame(columns=['rmse', 'mape'])

#Create cross-validation splits and run baseline on each
for i, (train_index, val_index) in enumerate (sws.split(testr)):
    # Storing windows in our window function
    train_df = testr.iloc[train_index]
    val_df = testr.iloc[train_index[-1]+1:val_index[0]+1]
    
    # Creating a baseline as the last week of each train window
    baseline = train_df.iloc[-168:].y.mean()
    #Store as predicted values
    val_df['yhat'] = baseline

    #Get model performance
    rmse1 = sqrt(mean_squared_error(val_df.y[:24], val_df.yhat[:24]))
    mape1 = wr.mean_absolute_percentage_error(val_df.y[:24], val_df.yhat[:24])
    rmse3 = sqrt(mean_squared_error(val_df.y, val_df.yhat))
    mape3 = wr.mean_absolute_percentage_error(val_df.y, val_df.yhat)

    #store baseline performance in df
    perf1 = {'rmse' : rmse1, 'mape':mape1}
    perf3 = {'rmse' : rmse3, 'mape':mape3}     
    baseline_test_df_1d = baseline_test_df_1d.append(perf1,ignore_index=True)
    baseline_test_df_3d = baseline_test_df_3d.append(perf3,ignore_index=True)

In [31]:
wr.print_model_results('Test Baseline',baseline_test_df_1d,baseline_test_df_3d)

Test Baseline model performance:
1 day rmse: 1685.0 MW
1 day mape: 11.9%
3 day rmse: 2889.0 MW
3 day mape: 18.8%


In [ ]:
#refit same model as above
m.fit(testr)
#Perform cross-validation
df_cv_test = cross_validation(m, initial='1461 days', period='36 days', horizon = '3 days',disable_tqdm=True) 
#Get performance metrics
df_test_1d = performance_metrics(df_cv_test, rolling_window=.33, metrics=metrics)
df_test_3d = performance_metrics(df_cv_test, rolling_window=1, metrics=metrics)

#print model results
wr.print_model_results('Tuned Prophet Model',df_test_1d,df_test_3d)

Recommendations
- Review acquisition/prep - find more reliable source of load data. Likely need to look at how it is written
- Use previous weather forecasts to actual weather and difference in load to create uncertainty bands
- Do more day type patterning for holidays
- Run an LSTM model.  Better able to use more historical data for day type patterning, with short term historical data for time series forecasting

### REMOVE BELOW IF ALL WENT WELL

In [19]:
# Setup for the implementation of the SlidingWindowSplitter
from sktime.forecasting.model_selection import SlidingWindowSplitter
window_length = 1461*24
fh = 24*3
step_length=36*24
#Create Splitter object
sws = SlidingWindowSplitter(window_length=window_length, fh=fh, step_length=step_length)

In [24]:
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
# #PREP Holidays
# #create calendar object
# cal = USFederalHolidayCalendar()
# #get holidays as list of dates
# train_holidays = cal.holidays(start=trainr.ds.min(),end=trainr.ds.max())

# # Transition to dataframe with holiday, ds columns
# holiday_df = pd.DataFrame(trainr.ds)
# #For each datetime, get if it lands on a holiday
# holiday_df['holiday'] = holiday_df.ds.dt.date.astype(str).isin(train_holidays.astype(str)).astype(int)
# #Filter down to just the holidays
# only_holidays = holiday_df[holiday_df.holiday==1]
# #Convert that column to string
# only_holidays.holiday = only_holidays.holiday.astype(str)